In [2]:
import pandas as pd

In [4]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [8]:
raw_csv_data = pd.read_csv(filepath_or_buffer="Absenteeism_data.csv");raw_csv_data.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [10]:
df = raw_csv_data.copy();df.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         700 non-null    int64  
 1   Reason for Absence         700 non-null    int64  
 2   Date                       700 non-null    object 
 3   Transportation Expense     700 non-null    int64  
 4   Distance to Work           700 non-null    int64  
 5   Age                        700 non-null    int64  
 6   Daily Work Load Average    700 non-null    float64
 7   Body Mass Index            700 non-null    int64  
 8   Education                  700 non-null    int64  
 9   Children                   700 non-null    int64  
 10  Pets                       700 non-null    int64  
 11  Absenteeism Time in Hours  700 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 65.8+ KB


#### drop ID

* The id variable is a categorical nominal variable. Since it does not explain the target variable and does not contribute to predicting the target variable in the analytical study to be conducted, we remove it from the dataset.

In [15]:
df = df.drop(["ID"], axis = 1)

#### Reason for Absence

- Since it is a categorical nominal variable, we perform a dummy variable transformation.

In [21]:
df["Reason for Absence"].unique()

array([26,  0, 23,  7, 22, 19,  1, 11, 14, 21, 10, 13, 28, 18, 25, 24,  6,
       27, 17,  8, 12,  5,  9, 15,  4,  3,  2, 16], dtype=int64)

In [23]:
len(df["Reason for Absence"].unique())

28

In [25]:
print(sorted(df["Reason for Absence"].unique()), end = " ")

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28] 

In [27]:
reason_columns = pd.get_dummies(data = df["Reason for Absence"], drop_first=True, dtype="int")

In [29]:
reason_columns.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [31]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [33]:
multi_coll = pd.concat([df.iloc[:, [2, 3, 4, 5, 6, 7, 8, 9]], reason_columns], axis = 1)
X = add_constant(multi_coll)

vif = pd.DataFrame()
vif.index = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])];vif.T

,const,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28
VIF,228.767221,1.834847,1.399365,1.709782,1.104407,1.795419,1.455824,1.444185,1.469885,1.428816,1.041445,1.030987,1.063891,1.093471,1.171735,1.350075,1.139144,1.119315,1.550569,1.647221,1.248955,2.306276,1.450394,1.062409,1.100373,1.039522,1.557496,1.894225,1.184988,1.870847,4.054673,1.090788,1.767295,1.782909,2.9303,3.480575


<ul><li><strong>VIF close to 1</strong>: No multicollinearity (ideal case).</li><li><strong>VIF &gt; 5</strong>: Moderate multicollinearity (should be monitored).</li><li><strong>VIF &gt; 10</strong>: High multicollinearity (problematic).</li></ul>

In [35]:
df = df.drop(["Reason for Absence"], axis = 1)

#### grouping reason_columns

* We group the dummy features to reduce their number and ensure that similar reasons are consolidated under general categories.

In [40]:
reason_type1 = reason_columns.loc[:, 1:14].max(axis = 1)
reason_type2 = reason_columns.loc[:, 15:17].max(axis = 1)
reason_type3 = reason_columns.loc[:, 18:21].max(axis = 1)
reason_type4 = reason_columns.loc[:, 22:].max(axis = 1)

#### concatenate the reason_types and df

In [76]:
df = pd.concat([df, reason_type1, reason_type2, reason_type3, reason_type4], axis= 1);df.head()

,reason_1,reason_2,reason_3,reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,0,1,2,3
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1


In [44]:
df.columns = ['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours',"reason_1", "reason_2", "reason_3", "reason_4"]
df.columns

Index(['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children',
       'Pets', 'Absenteeism Time in Hours', 'reason_1', 'reason_2', 'reason_3',
       'reason_4'],
      dtype='object')

In [46]:
df = df[["reason_1", "reason_2", "reason_3", "reason_4", 'Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']]

In [48]:
df_reason_mod = df.copy()

#### Date Column

In [50]:
df_reason_mod["Date"] = pd.to_datetime(df_reason_mod["Date"], format = "%d/%m/%Y")

#### Extracting month and weekday value

In [52]:
df_reason_mod.insert(5, column="Month", value = list(map(lambda x: x.month, df_reason_mod["Date"])))

In [54]:
def week_day(value):
    return value.weekday() + 1

df_reason_mod.insert(6, column = "Day of the Week", value = df_reason_mod["Date"].apply(week_day))
assert df_reason_mod["Date"].apply(week_day).max() <= 7 and  df_reason_mod["Date"].apply(week_day).min() >= 1, "values should be between 1-7 "

In [56]:
df_reason_mod.head()

,reason_1,reason_2,reason_3,reason_4,Date,Month,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,2015-07-07,7,2,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,2015-07-14,7,2,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,2015-07-15,7,3,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,2015-07-16,7,4,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,2015-07-23,7,4,289,36,33,239.554,30,1,2,1,2


In [58]:
df_reason_date_mod = df_reason_mod.copy()

In [60]:
df_reason_date_mod.drop(["Date"], axis = 1, inplace = True)

In [62]:
df_reason_date_mod.head()

,reason_1,reason_2,reason_3,reason_4,Month,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,2,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,7,2,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,7,3,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,7,4,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,7,4,289,36,33,239.554,30,1,2,1,2


In [64]:
df_reason_date_mod["Daily Work Load Average"].mean()

271.8017742857143

In [66]:
df_reason_date_mod["Education"].unique()

array([1, 3, 2, 4], dtype=int64)

#### Education Column
- converting to separete category

In [68]:
df_reason_date_mod["Education"].value_counts().to_frame().T.rename(columns = {1:"high_school", 2: "graduate", 3: "postgraduate", 4: "master or a doctor"})

Education,high_school,postgraduate,graduate,master or a doctor
count,583,73,40,4


In [70]:
df_reason_date_mod["Education"] = df_reason_date_mod["Education"].replace(to_replace={1:0, 2:1, 3:1, 4:1})
assert df_reason_date_mod["Education"].isin([0, 1]).all(), "values should one or zero"

In [72]:
df_reason_date_mod["Education"].value_counts()

Education
0    583
1    117
Name: count, dtype: int64

In [74]:
df_preprocessed =  df_reason_date_mod.copy()

In [231]:
df_preprocessed.to_csv("Absenteeism_preprocessed.csv", index = False )